\---S1---/\---S2---/\---S3---/
WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
____________\---V1---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

             \---S1---/\---S2---/\---S3---/
             WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
______________________\---V2---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


A = audio sample
V1, V2, etc = viseme 1, viseme 2, etc (aka "frame", but not in the MFCC sense)
W = a window of audio samples that will be used as input (aka "frame" in the MFCC sense. to avoid confusion, we refer to this as the "window" and the viseme as the "frame")
S1, S2, S3 = STFT of a window 
num_frames == num_windows

audio_bins_per_window = the number of S per W
samples_per_bin = the length of each S (i.e. number of As)
bin_hop_length = the number of A between S1 and S2 (in the picture above, hop_length == len(S1) == len(S2) == samples_per_bin

in practice, we calculate as such:
          /-----------V2--------------\
/--------------V1------------\
\---S1---/\---S2---/\---S3---/\---S4---/\---S5---/\---S6---/
_______________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

then at V1, we take {S1,S2,S3}, at V2 we take {S4,S5,S6}, etc
The STFT 
TFT hop length should therefore equal the length of a single viseme frame, in samples

In [1]:
from kaldi_mels import kaldi_mels
from models import BiLSTMModel
from dataset import preprocess_viseme, VisemeDataset

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from collections import OrderedDict
import pandas as pd
import math

import numpy as np
import soundfile as sf
import scipy
from scipy import fft
import functools
from torch import nn
import librosa
import math
import torchaudio 
import math
import yaml
from tensorflow_tts.inference import AutoConfig
import json

import torch.onnx 
import onnx
from onnx_tf.backend import prepare
import onnxruntime

torch.__version__

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Using {} device'.format(device))

from config import VisemeModelConfiguration
config = VisemeModelConfiguration(batch_size=20)
config.save("output/viseme_model.json")
config.model_config

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/home/hydroxide/.local/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's

Using cuda device


{'modelPath': 'bilstm.tflite',
 'targetNames': ['A37_Mouth_Close',
  'A29_Mouth_Funnel',
  'A30_Mouth_Pucker',
  'Merged_Open_Mouth'],
 'frameRate': 4.000667111407605,
 'sampleRate': 22050,
 'paddedAudioLength': 5,
 'frameLength': 5511.580790395198,
 'seqLength': 20,
 'windowLength': 44100,
 'stftFramesPerWindow': 52,
 'hopSize': 848,
 'numMels': 10,
 'fmin': 100,
 'fmax': 8000,
 'fftSize': 512}

In [2]:
from audio import pad_audio, load_and_pad_audio, coeffs_to_windows



def coeffs_to_windows(coeffs, config):
    output = np.zeros((config["seq_length"], config["stftFramesPerWindow"] *
        config["numMels"]))

    hop = int(config["stftFramesPerWindow"]/2)

    frameIdx = 0;

    for i in range(config["seq_length"]):

        startIdx = max(frameIdx-hop, 0)
        endIdx = min(frameIdx+hop, coeffs.shape[0])
        window = coeffs[startIdx:endIdx,:]

        if frameIdx - hop < 0:
            window = np.pad(window, [((hop*2) - window.shape[0], 0), (0,0)], constant_values=coeffs[0,0])
        elif frameIdx + hop > coeffs.shape[0]:
            window = np.pad(window, [(0,(hop*2) - window.shape[0]),(0,0)], constant_values=coeffs[0,0])
        #print(window.shape)
        output[i] = np.reshape(window, config["stftFramesPerWindow"]*config["numMels"])
        frameIdx += hop
    return output






config.model_config["window"] = "hanning"
#config.model_config["hopSize"] = 100
#tftts_mels("the_dog_barked_its_way2.wav", config.model_config)[0]



In [3]:


#base = pd.read_csv("data/training/speaker_3/MySlate_14_Nic_88.csv").head(1)

process_audio = functools.partial(kaldi_mels, 
                                          config=config.model_config)      

process_viseme = functools.partial(preprocess_viseme, 
                                   pad_len_in_secs=config.model_config["paddedAudioLength"], 
                                   blendshapes=config.sourceKeys, 
                                   source_framerate=config.source_config["framerate"],
                                   collapse_factor=int(config.source_config["framerate"] / config.model_config["frameRate"]),
                                  )

training_data = VisemeDataset("./data/training/speaker_3/", 
                              process_audio, \
                              process_viseme, 
                              num_ipa_symbols=157,
                             text_pad_len=config.text_pad_len)
test_data = VisemeDataset("./data/test/speaker_3/", 
                              process_audio, \
                              process_viseme,
                              num_ipa_symbols=157,
                              text_pad_len=config.text_pad_len
                             )
train_dataloader = DataLoader(training_data, batch_size=config.batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=config.batch_size, shuffle=True)



def preprocess_viseme(path,
                      pad_len_in_secs=None,
                      collapse_factor=None,
                      blendshapes=None,
                      source_framerate=None):

    csv = pd.read_csv(path)
    # remove the Timecode column because we don't need it
    columns = list(csv.columns)
    columns.remove("Timecode")
    if blendshapes is not None:
        columns = [c for c in columns if c in blendshapes]
    csv = csv[columns]
    csv.reset_index()

    # pad the visemes to the intended length, using the first row as our base
    pad_len = int(pad_len_in_secs * source_framerate)
    
    if csv.shape[0] < pad_len:
        pad_indices = csv.shape[0]
        csv = csv.append([csv.head(1)] * (pad_len - csv.shape[0]),ignore_index=True)
        csv.iloc[pad_indices:] = 0
    else:
        csv = csv.iloc[:pad_len]
        #print("Visemes exceeded max length, truncate?")
    
    # reduce the framerate by taking the mean of every X frames
    if collapse_factor is not None:
        i = 0
        while i < csv.shape[0]:
            csv.iloc[i] = csv.iloc[i:i+collapse_factor].mean()
            i += collapse_factor
        csv = csv.iloc[::collapse_factor]
    return csv





df = preprocess_viseme(
         "data/training/speaker_3/20211003_MySlate_27_27.csv", 
#     #"data/training/speaker_3/MySlate_14_Nic_88.csv",
         pad_len_in_secs=config.model_config["paddedAudioLength"], 
         source_framerate=config.source_config["framerate"],
         collapse_factor=int(config.source_config["framerate"] / config.model_config["frameRate"]),
)

# #df = df[list(config.source_config["mappings"].keys())]#df[sourceKeys]
# #df.columns = list(config.source_config["mappings"].keys()) #model_config["targetNames"]

# #df.to_csv("output/predicted.csv", index=False)
df["JawOpen"]

0      0.000000
14     0.000093
28     0.002042
42     0.002919
56     0.075545
70     0.321100
84     0.131893
98     0.591477
112    0.069186
126    0.291685
140    0.205711
154    0.103206
168    0.653481
182    0.252912
196    0.010072
210    0.000000
224    0.000000
238    0.000000
252    0.000109
266    0.000000
280    0.000000
294    0.000000
Name: JawOpen, dtype: float64

In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, 
                 input_dim=None):
        super(TransformerBlock, self).__init__()
        
        self.attn = nn.MultiheadAttention(input_dim, 8, batch_first=True)
        
        self.norm1 = nn.LayerNorm(input_dim)
        
        self.ff1 = nn.Sequential(
            nn.Linear(input_dim, input_dim*4),
            nn.ReLU(),
            nn.Linear(input_dim*4, input_dim)            
        )
        
        self.norm2 = nn.LayerNorm(input_dim)
               
    def forward(self, feats):
        attn_out, _ = self.attn(feats, feats, feats)
        out = feats + attn_out
        out = self.norm1(out)
        f = self.ff1(out)
        f += out
        out = self.norm2(out)
        return out
                
        
class SelfAttnModel(nn.Module):
    def __init__(self, 
                 seq_length=None, 
                 input_dim=None, 
                 text_dim=None, 
                 text_embedding_dim=256,
                 text_padding_idx=None,
                 num_visemes=15):
        super(SelfAttnModel, self).__init__()
        self.text_embedding = nn.Sequential(
                nn.Embedding(text_dim,text_embedding_dim),                
        )
        self.text_attention = nn.MultiheadAttention(text_embedding_dim, 4, batch_first=True)
        
        self.audio_proj = nn.Sequential(
            nn.Linear(input_dim,input_dim*4),
            nn.ReLU(),
            nn.Linear(input_dim*4,input_dim)
        )
        
        self.t1 = TransformerBlock(input_dim)
        self.t2 = TransformerBlock(input_dim)
        self.t3 = TransformerBlock(input_dim)
        self.t4 = TransformerBlock(input_dim)

        self.dropout = torch.nn.Dropout(p=0.1)
        
        self.text_ff = nn.Sequential(
            nn.Linear(text_embedding_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, text_embedding_dim)            
        )
        
        self.linear_out = nn.Sequential(
            nn.Linear(input_dim, num_visemes)            
        )            
        
        self.tblock = nn.Sequential(
            self.t1,
            self.t2,
        #    self.t3,
         #   self.t4,
        )
        
    def forward(self, audio_feats, text_feats):
        a = self.tblock(self.audio_proj(audio_feats))
        return self.linear_out(a)

input_dim=1248
model = SelfAttnModel(
     input_dim=input_dim,
      text_embedding_dim=2048,
     num_visemes=len(config.model_config["targetNames"]),
     text_dim=training_data.num_ipa_symbols).to(device)


batch = iter(train_dataloader)
audio_features, text_features, num_padded, train_labels, train_files = next(batch, (None,None,None,None,None))
print(audio_features.size())

learning_rate = 0.0005
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

num_steps = 100000
print_loss_every = 50
eval_every = 100

accum_loss = 0

torch.Size([20, 20, 520])


In [5]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim=None, text_dim=None, hidden_size=512, num_visemes=4):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_dim, hidden_size, 1, bidirectional=True)#,proj_size=int(hidden_size/2))
        self.text_embedding = nn.Embedding(text_dim,hidden_size*2)
        
        self.attention = nn.MultiheadAttention(int(hidden_size)*2, 1, batch_first=True)
        
        self.text_attention = nn.MultiheadAttention(hidden_size*2, 1, batch_first=True)
        self.output_linear = torch.nn.Linear(int(hidden_size)*2, num_visemes)
        #self.output_linear = nn.ModuleList([torch.nn.Linear(hidden_size*2, 1) for v in range(num_visemes)])
        
    def forward(self, x, t):
        out_f, (h,c) = self.lstm(x)
        out_f, attn_output_weights = self.attention(out_f, out_f, out_f)
        out_t =self.text_embedding(t)
        out_t,_ = self.text_attention(out_t,out_t,out_t)
        out_f += out_t
        return self.output_linear(out_f)
        #return self.output_linear],dim=2),dim=3)

input_dim=520

model = BiLSTMModel(
    input_dim=input_dim,
    num_visemes=len(config.model_config["targetNames"]),
    hidden_size=128,
    text_dim=training_data.num_ipa_symbols
).to(device)


learning_rate = 0.00005
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

num_steps = 100000
print_loss_every = 500
eval_every = 1000

accum_loss = 0

In [6]:
batch = iter(train_dataloader)

audio_features, text_features, num_padded, train_labels, train_files = next(batch, (None,None,None,None,None))
train_labels = train_labels[:, :20,]
print(train_labels.size())
print(audio_features.size())


torch.Size([20, 20, 4])
torch.Size([20, 20, 520])


In [ ]:
for t in range(num_steps):
    optimizer.zero_grad()
        
    audio_features, text_features, num_padded, train_labels, train_files = next(batch, (None,None,None,None,None))
    
    if audio_features is None:
        batch = iter(train_dataloader)
        audio_features, text_features, num_padded, train_labels, _ = next(batch)

    train_labels = train_labels[:, :20,]

    preds = model(audio_features.to(device),text_features.to(device)) 
    
    train_labels = train_labels.to(device)
    
    
    loss = torch.nn.functional.mse_loss(preds, train_labels.to(device))   
    #loss = torch.nn.functional.huber_loss(preds, train_labels.to(device))
    loss.backward()
    optimizer.step()
    
    if len(preds.shape) < 3:
        preds = torch.unsqueeze(preds, 0)
    
    #for i in range(preds.shape[1] - 1):
    #    cosine_loss = torch.nn.functional.cosine_embedding_loss(
    #        preds[:,i,:], 
    #        preds[:,i+1,:], 
    #        (torch.ones(preds.shape[0])
    #    ).to(device))
    #    loss += cosine_loss
    
    accum_loss += loss.item()
    if t > 0 and t % print_loss_every == 0:
        print(f"Step {t} Avg loss: {accum_loss / print_loss_every}")
        accum_loss = 0
        


    if t > 0 and t % eval_every == 0:
        accum_loss = 0
        for audio_feats, text_feats, test_mask, test_labels, _ in iter(test_dataloader):
            text_feats = text_feats#[:,:35]
            test_labels = test_labels[:,:20]

            y = test_labels.to(device)
            preds = model(audio_feats.to(device), text_feats.to(device))
            #accum_loss += torch.nn.functional.mse_loss(preds, y).item()
            accum_loss += torch.nn.functional.huber_loss(preds, y).item()

#         print(f"Test loss {accum_loss}")
#         accum_loss = 0
    
#pred_probab = nn.Softmax(dim=1)(logits)
#y_pred = pred_probab.argmax(1)
#print(f"Predicted class: {y_pred}")199

Step 500 Avg loss: 0.014349785579368472
Step 1000 Avg loss: 0.01263991227094084
Step 1500 Avg loss: 0.012164079545997084
Step 2000 Avg loss: 0.0104728421587497
Step 2500 Avg loss: 0.010174975720234215
Step 3000 Avg loss: 0.008762580586597324
Step 3500 Avg loss: 0.008707860549911857
Step 4000 Avg loss: 0.007490967528894544
Step 4500 Avg loss: 0.007644269648939371
Step 5000 Avg loss: 0.006536910380236805
Step 5500 Avg loss: 0.0068740289038978514
Step 6000 Avg loss: 0.005814374113455415
Step 6500 Avg loss: 0.006138323207385838
Step 7000 Avg loss: 0.005237848003394901
Step 7500 Avg loss: 0.0056132680959999565
Step 8000 Avg loss: 0.0047075663055293265
Step 8500 Avg loss: 0.0051784727447666224
Step 9000 Avg loss: 0.004381308658979833
Step 9500 Avg loss: 0.004891385016962886
Step 10000 Avg loss: 0.004041455454658717
Step 10500 Avg loss: 0.004642288046889007
Step 11000 Avg loss: 0.003762567857746035
Step 11500 Avg loss: 0.004338051002472639
Step 12000 Avg loss: 0.0035140572586096824
Step 12500

In [ ]:
Step 500 Avg loss: 0.04806631846912205
Step 1000 Avg loss: 0.002916151871555485
Step 1500 Avg loss: 0.004990221144282259
Step 2000 Avg loss: 0.0020618126165063587
Step 2500 Avg loss: 0.0005171663404908031
Step 3000 Avg loss: 0.0010838660373847233
Step 3500 Avg loss: 0.0002996790710603818
Step 4000 Avg loss: 0.0010451974497264018
Step 4500 Avg loss: 0.0005168943077296717
Step 5000 Avg loss: 0.00038338914602172735
Step 5500 Avg loss: 0.00023643585942409117
Step 6000 Avg loss: 0.0003196997099294094
Step 6500 Avg loss: 0.00030932931147981433
Step 7000 Avg loss: 0.00029204771798686124
Step 7500 Avg loss: 7.222784709119878e-05
Step 8000 Avg loss: 0.00017775368443471962
Step 8500 Avg loss: 0.00019257872350499384
Step 9000 Avg loss: 0.00022026305821873393
Step 9500 Avg loss: 6.478172176593944e-05
Step 10000 Avg loss: 0.00013493354754427856
Step 10500 Avg loss: 5.010930418575299e-05
Step 11000 Avg loss: 0.00011357640103597077
Step 11500 Avg loss: 8.715347881980051e-05

In [ ]:
audio_features[0,0:10]

In [ ]:
train_files


In [ ]:
print(train_labels.size())
train_labels[0]


In [ ]:
#torch.set_printoptions(profile="full")


model(audio_features.to(device), text_features.to(device))[0]

#train_labels
#audio_features[0,3,:10]
#audio_features.size()

In [ ]:
model = torch.load("output/conv1d.torch").to(device)
model.eval()
fft, num_samples = process_audio("the_dog_barked_its_way2.wav")
fft = torch.unsqueeze(fft,dim=0)
print(fft.size())

ipa_indices = [[int (x) for x in "158 69 76 157 31 108 68 157 29 0 115 98 118 157 70 118 116 157 139 34 157 23 118 157 40 137 157 32 40 157 115 131 100 159 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160 160".split(" ")]]
ipa_indices = torch.tensor(ipa_indices,dtype=torch.int32)
model(fft.to(device), ipa_indices.to(device))


In [ ]:
header = "Timecode,BlendShapeCount,eyeBlinkRight,eyeLookDownRight,eyeLookInRight,eyeLookOutRight,eyeLookUpRight,eyeSquintRight,eyeWideRight,eyeBlinkLeft,eyeLookDownLeft,eyeLookInLeft,eyeLookOutLeft,eyeLookUpLeft,eyeSquintLeft,eyeWideLeft,jawForward,jawRight,jawLeft,jawOpen,mouthClose,mouthFunnel,mouthPucker,mouthRight,mouthLeft,mouthSmileRight,mouthSmileLeft,mouthFrownRight,mouthFrownLeft,mouthDimpleRight,mouthDimpleLeft,mouthStretchRight,mouthStretchLeft,mouthRollLower,mouthRollUpper,mouthShrugLower,mouthShrugUpper,mouthPressRight,mouthPressLeft,mouthLowerDownRight,mouthLowerDownLeft,mouthUpperUpRight,mouthUpperUpLeft,browDownRight,browDownLeft,browInnerUp,browOuterUpRight,browOuterUpLeft,cheekPuff,cheekSquintRight,cheekSquintLeft,noseSneerRight,noseSneerLeft,tongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll".split(',')

In [ ]:
audio_feats, text_feats, _, _, files = next(iter(test_dataloader))
export_y_batch = model(audio_feats.to(device), text_feats.to(device))
export_y = export_y_batch[0,:,:]



selected_output_indices = [header.index(x[0].lower() + x[1:]) for x in source_config["mappings"].keys()]
num_visemes = len(source_config["mappings"])

# the prediction in LiveLink Face format
with open("output/prediction.csv", "w") as outfile:
    outfile.write(",".join(header) + "\n")
    timer_ms = 0
    for t in range(export_y.shape[1]):
        output = [str(0)] * len(header)
        second = str(int(timer_ms // 1000)).zfill(2)
        frame = (timer_ms % 1000) * model_config["frameRate"] / 1000
        output[0] = f"00:00:{second}:{frame}"
        for viseme in range(num_visemes): 
            output[selected_output_indices[viseme]] = str(export_y[viseme,t].item())
        timer_ms += (1 / model_config["frameRate"]) * 1000
        outfile.write(",".join(output) + "\n")
        
# the prediction in App format
with open("output/prediction_app.csv", "w") as outfile:
    outfile.write(",".join(model_config["targetNames"]) + "\n")
    for t in range(export_y.shape[0]):
        for v in range(export_y.shape[1]):
            outfile.write(str(export_y[t,v].item()))
            outfile.write(",")
        outfile.write("\n")

In [ ]:
new_header = "Timecode,BlendShapeCount,EyeBlinkLeft,EyeLookDownLeft,EyeLookInLeft,EyeLookOutLeft,EyeLookUpLeft,EyeSquintLeft,EyeWideLeft,EyeBlinkRight,EyeLookDownRight,EyeLookInRight,EyeLookOutRight,EyeLookUpRight,EyeSquintRight,EyeWideRight,JawForward,JawRight,JawLeft,JawOpen,MouthClose,MouthFunnel,MouthPucker,MouthRight,MouthLeft,MouthSmileLeft,MouthSmileRight,MouthFrownLeft,MouthFrownRight,MouthDimpleLeft,MouthDimpleRight,MouthStretchLeft,MouthStretchRight,MouthRollLower,MouthRollUpper,MouthShrugLower,MouthShrugUpper,MouthPressLeft,MouthPressRight,MouthLowerDownLeft,MouthLowerDownRight,MouthUpperUpLeft,MouthUpperUpRight,BrowDownLeft,BrowDownRight,BrowInnerUp,BrowOuterUpLeft,BrowOuterUpRight,CheekPuff,CheekSquintLeft,CheekSquintRight,NoseSneerLeft,NoseSneerRight,TongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll"
remap = {h:(h[0].lower() + h[1:]) if h not in ["Timecode","BlendShapeCount","HeadYaw","HeadPitch","HeadRoll","LeftEyeYaw","LeftEyePitch","LeftEyeRoll","RightEyeYaw","RightEyePitch","RightEyeRoll"]  else h for h in new_header.split(",") }
for oh in remap.values():
    if oh not in header:
        print(oh)

def new_to_old(csv_df):
    return csv_df.rename(columns=remap)
df = preprocess_viseme("data/training/speaker_3/MySlate_22_Nic_ps-15_20.csv", pad_len_in_secs=model_config["paddedAudioLength"], 
                                   target_framerate=model_config["frameRate"])
df = new_to_old(df)
#cols = [c for c in df.columns if c not in ["jawOpen", "mouthClose", "Timecode"]]
#df[cols] = 0
df = df[df["Timecode"] != 0]
df.to_csv("output/predicted.csv", index=False)
#new_to_old(df)[header].to_csv("original.csv", index=False)
df

In [ ]:
seq_length = audio_features.size()[1]
torch.save(model, "output/bilstm.torch")
model = torch.load("output/bilstm.torch",map_location=torch.device('cpu'))

model.eval() 

dummy_audio_input = torch.randn(1, seq_length, input_dim, requires_grad=False)  
print(dummy_audio_input)
dummy_text_input = torch.zeros(1, seq_length, requires_grad=False,dtype=torch.int)
print(dummy_text_input)

# Export the model   
torch.onnx.export(model,         # model being run 
     (dummy_audio_input,dummy_text_input),      # model input (or a tuple for multiple inputs) 
     "output/bilstm.onnx",       # where to save the model  
     export_params=True,  # store the trained parameter weights inside the model file 
     opset_version=10,    # the ONNX version to export the model to 
     do_constant_folding=True,  # whether to execute constant folding for optimization 
     input_names = ['audio_feats', 'text_feats'],   # the model's input names 
     output_names = ['modelOutput'], # the model's output names ,

) 

model_onnx = onnx.load('output/bilstm.onnx')

tf_rep = prepare(model_onnx)
tf_rep.export_graph('./output/tf_model')

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("./output/tf_model")
print("Built converter")

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=False
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_enable_resource_variables = True

converter.experimental_new_converter =True
tflite_model = converter.convert()
print("Converted")

# Save the model
with open("./output/bilstm.tflite", 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="output/bilstm.tflite")
interpreter.allocate_tensors()
    
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
#input_shape = input_details[0]['shape']
#output_details
input_details

In [ ]:
csv = pd.read_csv("data/training/speaker_1/20210824_1/61.csv")
columns = [x for x in list(csv.columns) if "Eye" not in x]
columns.remove("Timecode")
columns.remove("BlendShapeCount")
csv[columns].var().sort_values()
#df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=pad_len_in_secs, 
#                                   resample_to=target_framerate, blendshapes=["MouthClose","MouthFunnel"])
#df.shape
#[df.iloc[0]["EyeLookInLeft"]]
    #csv[columns] = pd.np.digitize(csv[columns], np.linspace(0,1,11))
    
    #split = csv["Timecode"].str.split(':')
    #minute = split.str[1].astype(int)
    #second = split.str[2].astype(int)
    #frame = split.str[3].astype(float)
    #minute -= minute[0]
    #ms
    #step = minute * 60 + second
    #csv["step"] = step
    #return csv.drop_duplicates(["step"])[["step", "MouthClose","MouthFunnel","MouthPucker","JawOpen"]]
    
# if we want to use softmax across each blendshape as a one-hot
    #return np.reshape(vals, (vals.shape[0], vals.shape[1], 1))
    #one_hot = np.zeros((vals.shape[0], vals.shape[1], 11, 1))
    #oh = np.eye(11)
    #for row in range(vals.shape[0]):
    #    for t in range(vals.shape[1]):
    #        one_hot[row, :, :, 0] = np.eye(11)[int(vals[row,t])-1]
    #return one_hot